In [ ]:
#| default_exp util

In order to avoid `tqdm`'s experimental warning

In [ ]:
#| include: false
import warnings

In [ ]:
warnings.filterwarnings("ignore", message='Using `tqdm.autonotebook.tqdm` in notebook mode')

In [ ]:
#| export
import pathlib
import re
import sys

import numpy as np
import IPython.display
import ruamel.yaml
import yaml
import pandas as pd
from pandas.core.accessor import _register_accessor as register_accessor

import gift_wrapper.core
import gift_wrapper.image

/home/manu/mambaforge/envs/py2gift/lib/python3.10/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Rendering latex in a notebook

Convenience function to print $\LaTeX$ text in Jupyter Notebook **code** (as opposed to *markdown*) cells. The purpose is to print $\LaTeX$ content that is only known at *run time*. Font size of every formula is increased (to `\Large`).

In [ ]:
#| export
def render_latex(
    text: str # Input text
) -> str: # Markdown text
    "Returns latex-aware markdown text"
    
    return IPython.display.Markdown(re.sub(r'\$([^\$]*)\$', '$' + '\\\Large ' + r'\1' + '$', text))

In [ ]:
render_latex('Bla blah...')

Bla blah...

In [ ]:
render_latex('Variable $A$')

Variable $\Large A$

## Accessors

[Pandas](https://pandas.pydata.org/)'s [accessors mechanishm](https://pandas.pydata.org/docs/reference/api/pandas.api.extensions.register_dataframe_accessor.html) is leveraged here. In order to do that, first Pandas' `_register_accessor` method was *aliased* above as simply `register_accessor`. Then a new *base* class, `AccessorEndowedClass`, from which any class exposing an accessor must inherit, is created. The only thing it does is adding a *class* attribute that is expected by Pandas' `_register_accessor` method.

In [ ]:
#| export
class AccessorEndowedClass:
    
    _accessors = set()

For instance

In [ ]:
class Car(AccessorEndowedClass):
    
    def __init__(self, weight):
        
        self.weight = weight

@register_accessor('seat', Car)
class SeatAccessor:
    
    def __init__(self, car: Car):
        
        self._confortable: bool = True
    
    @property
    def confortable(self):
        
        return self._confortable

Accessing the class itself

In [ ]:
car = Car(1600)
car.weight

1600

Accessing the *appended* accessor

In [ ]:
car.seat.confortable

True

## Assorted

A function to turn an integer into a roman number. Adapted from [here](https://www.w3resource.com/python-exercises/class-exercises/python-class-exercise-1.php).

In [ ]:
#| export
def int_to_roman(
    num: int # Input
) -> str: # Roman number for the input
    "Returns an integer number in roman format"
    
    val = [
        1000, 900, 500, 400,
        100, 90, 50, 40,
        10, 9, 5, 4,
        1
    ]
    syb = [
        "M", "CM", "D", "CD",
        "C", "XC", "L", "XL",
        "X", "IX", "V", "IV",
        "I"
    ]
    roman_num = ''
    i = 0
    while num > 0:
        for _ in range(num // val[i]):
            roman_num += syb[i]
            num -= val[i]
        i += 1
    return roman_num

assert int_to_roman(12) == 'XII'
assert int_to_roman(9) == 'IX'

In [ ]:
print(int_to_roman(53))

LIII


## Files

### Writing a dictionary into a YAML file

A function to write a `dict`ionary to a YAML file with some formatting.

In [ ]:
#| export
def dict_to_yaml(
    d: dict, # Input
    output_file: str | pathlib.Path # Ouput
) -> None:
    "Writes a dictionary in a YAML file"
    
    yaml = ruamel.yaml.YAML()
    yaml.indent(sequence=4, offset=2)

    with open(output_file, 'w') as f:

        yaml.dump(d, f)

In [ ]:
yaml_file = '_test.yaml'
d = {'foo': 1, 'ouch': 'psss'}
dict_to_yaml(d, yaml_file)

In [ ]:
%cat {yaml_file}

foo: 1
ouch: psss


### Extracting a dictionary from a YAML file

In [ ]:
#| export
def yaml_to_dict(
    file: str | pathlib.Path # Input file
) -> dict: # Output
    "Reads a dictionary from a YAML file"

    with open(file) as f:

        d = yaml.load(f, Loader=yaml.FullLoader)
        
    return d

In [ ]:
yaml_to_dict(yaml_file)

{'foo': 1, 'ouch': 'psss'}

In [ ]:
%rm {yaml_file}

## Writing an input file for gift-wrapper

In [ ]:
#| export
def write_multiple_categories(
    category_questions: dict[str, list[dict]], # Every key is the name of a category, and every value is a list of questions (every question is itself a dictionary)
    pictures_base_directory: str, # The "pictures base directory" parameter that must be passed to `gift-wrapper`
    output_file: str | None = 'out.yaml' # Output file
) -> None:
    "Writes a file suitable as input to `gift-wrapper`"

    file = dict()
    file['pictures base directory'] = pictures_base_directory
    file['categories'] = []

    for category_name, questions in category_questions.items():

        file['categories'].append({'name': category_name, 'questions': questions})
    
    dict_to_yaml(file, output_file)

We build a dictionary with questions belonging different categories. This might be read from a *YAML* file.

In [ ]:
category_questions_example = {
    ('Category A', 'Category B'): [
        {
            'class': 'MultipleChoice',
            'statement': 'Compute the entropy....\n',
            'feedback': 'We just need to...\n',
            'time': '3',
            'answers': {
                'perfect': '1',
                'wrong': ['0', '2']
            },
            'name': 'Entropy of a random variable I'
        }
    ],
    'Category C': [
        {
            'class': 'Numerical',
            'statement': 'Compute the average....\n',
            'feedback': 'In order to...\n',
            'time': '3',
            'solution': {
                'value': 3.1,
                'error': '10%'
            }, 
            'name': 'Mean energy I'
        }
    ]
}
category_questions_example

{('Category A',
  'Category B'): [{'class': 'MultipleChoice',
   'statement': 'Compute the entropy....\n',
   'feedback': 'We just need to...\n',
   'time': '3',
   'answers': {'perfect': '1', 'wrong': ['0', '2']},
   'name': 'Entropy of a random variable I'}],
 'Category C': [{'class': 'Numerical',
   'statement': 'Compute the average....\n',
   'feedback': 'In order to...\n',
   'time': '3',
   'solution': {'value': 3.1, 'error': '10%'},
   'name': 'Mean energy I'}]}

In [ ]:
output_file = '_output.yaml'
write_multiple_categories(category_questions_example, 'pics', output_file)

In [ ]:
%cat {output_file}

pictures base directory: pics
categories:
  - name:
      - Category A
      - Category B
    questions:
      - class: MultipleChoice
        statement: "Compute the entropy....\n"
        feedback: "We just need to...\n"
        time: '3'
        answers:
          perfect: '1'
          wrong:
            - '0'
            - '2'
        name: Entropy of a random variable I
  - name: Category C
    questions:
      - class: Numerical
        statement: "Compute the average....\n"
        feedback: "In order to...\n"
        time: '3'
        solution:
          value: 3.1
          error: 10%
        name: Mean energy I


In [ ]:
!rm {output_file}

## Question-related

### Naming different versions (instances) of a question

Adds a name to every question in a list based on a pattern. It is meant to distinctly name different versions of the same question. Notice that, for this function, a *question* is simply a dictionary (no checks are performed).

In [ ]:
#| export
def add_name(
    questions: list[dict], # List of questions; every question is a dictionary
    base_name: str # All the questions will be given this name and a different (Roman) number
) -> list[dict]: # List with the same questions after adding the corresponding name to each one
    "Adds a name to every question based on a pattern"

    res = []

    for i_q, q in enumerate(questions):

        res.append({**q, 'name': f'{base_name} {int_to_roman(i_q + 1)}'})

    return res

assert add_name([{'k1': 'aa', 'k2': 1}, {'k3': 'pi', 'foo': 'variance'}], 'Viterbi') == [
    {'k1': 'aa', 'k2': 1, 'name': 'Viterbi I'}, {'k3': 'pi', 'foo': 'variance', 'name': 'Viterbi II'}]

In [ ]:
add_name([{'k1': 'aa', 'k2': 1}, {'k3': 'pi', 'foo': 'variance'}], 'base')

[{'k1': 'aa', 'k2': 1, 'name': 'base I'},
 {'k3': 'pi', 'foo': 'variance', 'name': 'base II'}]

### Obtaining wrong solutions from the correct one

In [ ]:
#| export
def wrong_numerical_solutions_from_correct_one(
    solution: float, # The actual solution
    n: int, # The number of wrong solutions
    min_sep: float, # Minimum separation
    max_sep: float, # Maximum separation
    lower_bound: float | None = -np.inf, # A lower bound on the returned numbers
    upper_bound: float | None = np.inf, # A upper bound on the returned numbers
    precision: int | None = 4, # The number of decimal places
    to_str: bool | None = True, # If `True`, every element in the result will be converted to a string
    fixed_point_format: bool | None = False, # Only meaningful when `to_str` is `True`. In such case, if `True` a fixed-point format (`f`) is used regardless of the actual type
    bin_width: float | None = None, # The granularity on the answers: every one will be a multiple of this parameter
    unique: bool | None = False, # If `True`, all the answers will be different
    prng: np.random.RandomState | None = np.random.RandomState(42) # A pseudo-random numbers generator
) -> list[float] | list[str]: # The random wrong solutions
    "Generates random numerical wrong answers given the correct one"
    
    max_iterations = 1_000
    
    # in the current implementation all the answers are generated by pivoting around the solution; in order to prevent the correct solution being approximately in the middle, more wrong answers than requested are generated and, in the end, the requested number is randomly picked
    n_generated = n + 5
    
    assert (solution - min_sep > lower_bound) or (solution + min_sep < upper_bound)
    
    if bin_width:
        
        assert bin_width < (max_sep - min_sep)
    
    res = []
    
    for i in range(max_iterations):
        
        if bin_width:
        
            candidates = np.arange(min_sep, max_sep, bin_width)
            steps = prng.choice(candidates, size=2).round(precision)
        
        else:
            
            # one number to be added to the `solution` solution and one to be subracted
            steps = prng.uniform(min_sep, max_sep, size=2)

        next_values = [v.round(precision)
                       for v in [solution + steps[0], solution - steps[1]] if lower_bound < v < upper_bound]
        if unique:
            
            next_values = [e for e in next_values if e not in set(res)]
        
        res.extend(next_values)

        if len(res) >= n:
        
            break
            
    else:
        
        raise Exception(f"Parameters are too much constraining, can't find the solution after {max_iterations}")
    
    if to_str:
        
        format_specifier = f'.{precision}{"f" if fixed_point_format else "g"}'
        
#         res = [str(e) for e in res]
        res = [f'{e:{format_specifier}}' for e in res]
    
#     return res[:n]
    return prng.choice(res, n, replace=False).tolist()

In [ ]:
wrong_numerical_solutions_from_correct_one(solution=0.8, n=10, min_sep=0.1, max_sep=0.5, lower_bound=0.2, upper_bound=2, prng = np.random.RandomState(42))

['1.14',
 '1.193',
 '1.05',
 '0.9232',
 '0.3535',
 '0.4168',
 '0.4605',
 '0.3197',
 '0.9624',
 '0.6376']

Numbers instead of strings

In [ ]:
wrong_numerical_solutions_from_correct_one(
    solution=0.8, n=4, min_sep=0.2, max_sep=0.5, lower_bound=0.2, upper_bound=0.9, to_str=False,
    prng = np.random.RandomState(42))

[0.3148, 0.4204, 0.5532, 0.3401]

If the solution cannot be *anything*, you might want to pass a `bin_width` so that it doesn't immediately stand out from the wrong answers,

In [ ]:
wrong_numerical_solutions_from_correct_one(
    solution=0.8, n=8, min_sep=0.1, max_sep=2, lower_bound=0.2, upper_bound=2, to_str=False, bin_width=0.1,
    prng = np.random.RandomState(42))

[1.1, 1.5, 0.2, 0.6, 1.5, 1.2, 1.9, 1.9]

If duplicates must be avoided `unique` can be set to `True`,

In [ ]:
wrong_numerical_solutions_from_correct_one(
    solution=0.8, n=8, min_sep=0.1, max_sep=2, lower_bound=0.2, upper_bound=2, to_str=False, bin_width=0.1,
    unique=True, prng = np.random.RandomState(42))

[1.0, 1.2, 1.5, 0.7, 0.6, 0.2, 1.9, 1.1]

Lower and upper bound can be ommited

In [ ]:
wrong_numerical_solutions_from_correct_one(
    solution=0.8, n=8, min_sep=0.5, max_sep=10, to_str=False, bin_width=0.1, unique=True,
    prng = np.random.RandomState(42))

[6.4, -8.9, 9.5, -1.7, -6.8, 7.3, -8.3, 2.7]

When requesting strings, a floating point format can be enforced by using the parameter `fixed_point_format`,

In [ ]:
wrong_numerical_solutions_from_correct_one(
    solution=0.8, n=8, min_sep=0.5, max_sep=10, to_str=True, fixed_point_format=True, bin_width=0.1, unique=True,
    prng = np.random.RandomState(42))

['6.4000',
 '-8.9000',
 '9.5000',
 '-1.7000',
 '-6.8000',
 '7.3000',
 '-8.3000',
 '2.7000']

In [ ]:
#| hide
from nbdev.doclinks import nbdev_export

In [ ]:
#| hide
nbdev_export('20_util.ipynb')